In [1]:
import os
import hail as hl
%run {os.environ['NB_DIR']}/nb.py
%run $BENCHMARK_METHOD_DIR/common.py

sample_rate = .1
ds_name = DATASET_HM
min_ld = .1

# sample_rate = 1
# ds_name = DATASET_SIM
# min_ld = .1

ds_config = DATASET_CONFIG[ds_name]
ds_export_path = dataset_path(ds_name, sr=sample_rate)
hail_init()

Running on Apache Spark version 2.4.4
SparkUI available at http://2e4e0c6972f9:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.32-a5876a0a2853
LOGGING: writing to /home/eczech/repos/gwas-analysis/notebooks/benchmark/method/ld_prune/lsh/hail-20200223-2352-0.2.32-a5876a0a2853.log


In [2]:
mt = hl.import_plink(
    *plink_files(osp.dirname(ds_export_path), osp.basename(ds_export_path)),
    skip_invalid_loci=False,
    reference_genome=ds_config['reference_genome']
)
mt.count()

2020-02-23 23:52:59 Hail: INFO: Found 165 samples in fam file.
2020-02-23 23:52:59 Hail: INFO: Found 10451 variants in bim file.
2020-02-23 23:53:00 Hail: INFO: Coerced sorted dataset


(10451, 165)

In [3]:
# Run LD calculations with no effective radius limit (longest known genome < 150B bp)
ld = hl.ld_matrix(mt.GT.n_alt_alleles(), mt.locus, radius=150e9)
ld.shape

2020-02-23 23:53:01 Hail: INFO: Coerced sorted dataset
2020-02-23 23:53:02 Hail: INFO: Wrote all 3 blocks of 10451 x 165 matrix with block size 4096.
2020-02-23 23:53:02 Hail: INFO: Coerced sorted dataset


(10451, 10451)

In [6]:
lde = ld.entries()
assert lde.aggregate(hl.agg.counter(hl.is_nan(lde.entry))).get(True, 0) == 0, \
    'Some LD values are null which will make comparisons from the same source dataset incompatible'
if min_ld is not None:
    lde = lde.filter(lde.entry >= min_ld)
# Filter to upper triangle (no diagonal)
lde = lde.filter(lde.j > lde.i)
lde.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'i': int64 
    'j': int64 
    'entry': float64 
----------------------------------------
Key: ['i', 'j']
----------------------------------------


In [7]:
if ds_name == DATASET_SIM:
    import seaborn as sns
    sns.heatmap(lde.to_pandas().set_index(['i', 'j'])['entry'].unstack(), cmap='Spectral')

In [8]:
%%time
path = dataset_path(ds_name, sr=sample_rate, minld=min_ld) + '.parquet'
lde.to_spark().write.parquet(path, mode='overwrite')
!du -ch $path

68M	/home/eczech/data/gwas/benchmark/datasets/hapmap-sr=0.1-minld=0.1.parquet
68M	total
CPU times: user 37.4 ms, sys: 12.6 ms, total: 50 ms
Wall time: 32.3 s
